In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

from antra.antra import *
import json
import torch

from decode_graphical_models import *
from decode_abstract_models import *

In [2]:
path_to_data = "../../../data-files/gSCAN-Simple/data-compositional-splits.txt"
data_json = json.load(open(path_to_data, "r"))

In [15]:
NUM = 200
agent_positions_batch = []
target_positions_batch = []
target_commands = []
for ex in data_json["examples"]["train"]:
    target_commands += [ex["target_commands"]]
    situation_repr = ex['situation']
    agent = torch.tensor(
        (int(situation_repr["agent_position"]["row"]) * int(situation_repr["grid_size"])) +
        int(situation_repr["agent_position"]["column"]), dtype=torch.long).unsqueeze(dim=0)
    target = torch.tensor(
        (int(situation_repr["target_object"]["position"]["row"]) * int(situation_repr["grid_size"])) +
        int(situation_repr["target_object"]["position"]["column"]), dtype=torch.long).unsqueeze(dim=0)
    agent_positions_batch.append(agent)
    target_positions_batch.append(target)
    if len(agent_positions_batch) == NUM:
        break
agent_positions_batch = torch.cat(agent_positions_batch, dim=0)
target_positions_batch = torch.cat(target_positions_batch, dim=0)

In [16]:
input = {
    "agent_positions_batch": agent_positions_batch[0:1].unsqueeze(dim=-1),
    "target_positions_batch": target_positions_batch[0:1].unsqueeze(dim=-1),
}
input = GraphInput(input, batched=True, batch_dim=0, cache_results=False)

In [17]:
input

GraphInput({'agent_positions_batch': tensor([[8]]),
 'target_positions_batch': tensor([[14]])})

In [18]:
G = get_counter_compgraph(
    20,
    cache_results=False
)
#_ = G.compute(input)

In [19]:
a_map = {
    "NULL":0,
    "walk":4,
    "turn left":5,
    "turn right":3,
    0:"NULL",
    4:"walk",
    5:"turn left",
    3:"turn right"
}
for j in range(1):
    # print("\n\n\n", j, "a;oifwje;awfeoijawj \n\n\n")
    # print(goods[j]["command"])
    # print(goods[j]["target_commands"])
    for i in range(1, 21):
        # print(G.compute_node("x" + str(i), input)[j])
        # print(G.compute_node("y" + str(i), input)[j])
        # print(G.compute_node("o" + str(i), input)[j])
        # print(G.compute_node("a" + str(i), input)[j])
        a = a_map[G.compute_node("s" + str(i), input)[j][-1].tolist()]
        t_a = target_commands[j].split(",")
        if i <= len(t_a):
            # print(G.compute_node("s" + str(i), input)[j])
            print(a, t_a[i-1])
            assert a == t_a[i-1]
        else:
            assert a == "NULL"

turn right turn right
walk walk
